In [32]:
import ktwo19.io
%reload_ext autoreload
%autoreload 2
%pylab osx
from scipy import optimize

Populating the interactive namespace from numpy and matplotlib


In [10]:
reload(ktwo19.io)
df = ktwo19.io.load_table('rv-trend-removed')
clf()
plot(df.time,df.mnvel,'.')

excluding obs: rj197.149
MAP fit
parameter                     value      vary
per1                        7.91952      False
k1                          9.04338       True
per2                        11.9072      False
k2                          1.21229       True
tc1                         1980.38      False
secosw1                           0      False
sesinw1                           0      False
tc2                         1984.28      False
secosw2                           0      False
sesinw2                           0      False
dvdt                       0.015778       True
curv                              0      False
gamma_j                    -2.69249       True
jit_j                       7.13192       True

Priors
------
e1 constrained to be < 0.99
e2 constrained to be < 0.99
Bounded prior on jit_j, min=0.0, max=10.0

Zeroing out dvdt and trend
parameter                     value      vary
per1                        7.91952      False
k1                          9

In [22]:
import radvel.utils
from scipy import optimize

ti = linspace(1900,4000,10000)
P, post = radvel.utils.initialize_posterior('analysis/radvel/ktwo19_npl=2-cc.py')
res  = optimize.minimize(post.neglogprob_array, post.get_vary_params(), method='Nelder-Mead',) # MAP fit

print "MAP fit"
print post
post.params['dvdt'].value = 0 
post.params['gamma_j'].value = 0 

print "Zeroing out dvdt and trend"
print post
model = post.likelihood.model
rvtimes = post.likelihood.x # times where model is computed  
rvmod = model(rvtimes) # 
rawresid = post.likelihood.residuals() # array
rv_dt = rawresid + rvmod # residuals plus model without trend
rv_dt_err = post.likelihood.yerr
figure(figsize=(10,3) )
grid()
errorbar(rvtimes,rv_dt,yerr=rv_dt_err,fmt='.')
plot(ti, model(ti))

table = P.data.sort_values(by='time')
sidx = np.argsort(rvtimes)

assert np.allclose(table.time,rvtimes[sidx]), "Times must be sorted"

table['rv_dt'] = rv_dt[sidx]
table['rv_dt_err'] = rv_dt_err[sidx]
table = table.rename(columns={'mnvel':'rv','errvel':'rv_err','tel':'inst'})

#table = table.reset_index()
#table = table.sort_values(by='time')
table = table['time rv rv_err rv_dt rv_dt_err inst'.split()]
print table.head(10)

header = """\
% time rv rv_err rv_dt rv_dt_err inst    
% time: BJD - 2454833
% rv: stellar radial velocity (m/s)
% rv_err: uncertainty on (m/s)
% rv_dt: rv after removing gamma offsets between instruments
% rv_dt_err: uncertainty on rv_dt, jitter added in quadrature (m/s)
"""

csvfn = table.to_csv('paper/tab_rv.csv',index=False)
f = open('paper/tab_rv.tex','w')
g = open('paper/tab_rv.csv','w')
h = open('paper/tab_rv_stub.tex','w')

f.writelines(header)
g.writelines(header.replace('%','#'))
g.write("time,rv,rv_err,rv_dt,rv_dt_err,inst\n")
for i, row in table.iterrows():
    texstr = "{time:.6f} & {rv:.2f} & {rv_err:.2f} & {rv_dt:.2f} & {rv_dt_err:.2f} & {inst:s} \\\\ \n"
    f.write(texstr.format(**row))
    g.write("{time:.6f},{rv:.2f},{rv_err:.2f},{rv_dt:.2f},{rv_dt_err:.2f},{inst:s}\n".format(**row))
    if i < 10:
        h.write(texstr.format(**row))

f.close()
g.close()
h.close()

print rv_dt[-1]
table.tail()

excluding obs: rj197.149
MAP fit
parameter                     value      vary
per1                        7.91952      False
k1                          9.04338       True
per2                        11.9072      False
k2                          1.21229       True
tc1                         1980.38      False
secosw1                           0      False
sesinw1                           0      False
tc2                         1984.28      False
secosw2                           0      False
sesinw2                           0      False
dvdt                       0.015778       True
curv                              0      False
gamma_j                    -2.69249       True
jit_j                       7.13192       True

Priors
------
e1 constrained to be < 0.99
e2 constrained to be < 0.99
Bounded prior on jit_j, min=0.0, max=10.0

Zeroing out dvdt and trend
parameter                     value      vary
per1                        7.91952      False
k1                          9

,time,rv,rv_err,rv_dt,rv_dt_err,inst
46,3092.825866,-10.137460,2.283099,-10.137460,2.283099,j
47,3093.840536,8.498030,1.930745,8.498030,1.930745,j
48,3259.112368,15.107988,2.094515,15.107988,2.094515,j
49,3280.131860,14.551219,1.977589,14.551219,1.977589,j
50,3281.110002,12.981141,2.031842,12.981141,2.031842,j


In [13]:
ktwo19.io.load_table('transit-times')

AssertionError: Table transit-times not valid table name

In [33]:
import radvel.utils
ti = linspace(1900,4000,10000)
P, post = radvel.utils.initialize_posterior('analysis/radvel/ktwo19_npl=2-cc.py')
res  = optimize.minimize(post.neglogprob_array, post.get_vary_params(), method='Nelder-Mead',) # MAP fit

print "MAP fit"
print post
post.params['dvdt'].value = 0 
post.params['gamma_j'].value = 0 


# Full ephem
df = pd.read_excel('data/data.xlsx',sheet_name='transit-times')
df = df.dropna(subset=['inst'])
df[df.inst.str.contains('K2|Spitzer')]
df1 = df[df.i_planet==1]
df2 = df[df.i_planet==2]
ephem1 = polyfit(df1.i_epoch,df1.tc,1)
ephem2 = polyfit(df2.i_epoch,df2.tc,1)

post.params['per1'].value = ephem1[0]
post.params['tc1'].value = ephem1[1]
post.params['per2'].value = ephem2[0]
post.params['tc2'].value = ephem2[1]








excluding obs: rj197.149
MAP fit
parameter                     value      vary
per1                        7.91952      False
k1                          9.04338       True
per2                        11.9072      False
k2                          1.21229       True
tc1                         1980.38      False
secosw1                           0      False
sesinw1                           0      False
tc2                         1984.28      False
secosw2                           0      False
sesinw2                           0      False
dvdt                       0.015778       True
curv                              0      False
gamma_j                    -2.69249       True
jit_j                       7.13192       True

Priors
------
e1 constrained to be < 0.99
e2 constrained to be < 0.99
Bounded prior on jit_j, min=0.0, max=10.0



In [31]:
ephem = ktwo19.io.load_table('ephem-sinukoff16')
ephem


key
per1           7.91940
per1_err       0.00005
tc1         1980.38370
tc1_err        0.00030
dur1           3.50200
dur1_err       0.06300
per2          11.90715
per2_err       0.00150
tc2         1984.27550
tc2_err        0.00510
dur2           4.37100
dur2_err       1.00000
per3           2.50856
per3_err       0.00041
tc3         1975.92070
tc3_err        0.00860
dur3           2.17000
dur3_err       0.32800
Name: value , dtype: float64

In [37]:
ephem1[0] - ephem.per1


0.00092626281488872309

In [38]:
ephem2[0] - ephem.per2


-0.0091140789036305847

In [39]:
import jinja2

[autoreload of radvel failed: Traceback (most recent call last):
  File "/Users/petigura/anaconda2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name posterior
]
[autoreload of radvel failed: Traceback (most recent call last):
  File "/Users/petigura/anaconda2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name posterior
]
[autoreload of radvel failed: Traceback (most recent call last):
  File "/Users/petigura/anaconda2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name posterior
]


In [66]:

from jinja2 import Environment

Environment().from_string("{{ '\includegraphics[height=8.0in,width=6.0in,keepaspectratio]{'~title~'}' }}").render(title='Hello Gist from GutHub')


u'\\includegraphics[height=8.0in,width=6.0in,keepaspectratio]{Hello Gist from GutHub}'